In [1]:
import numpy as np
import pandas as pd

In [2]:
# https://www.kaggle.com/code/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif col != 't_dat':
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

# Step 1 - Preprocessing

In [4]:
input_path_prefix = "../../../DSCI-599/Final-Project/Data/"

print('-' * 80)
print('articles')
pre_articles = import_data(input_path_prefix + "articles.csv")
#pre_articles = pd.read_csv('Data/articles.csv')
print('-' * 80)
print('customers\'s memory usage will be better off without the memory optimization')
pre_customers = pd.read_csv(input_path_prefix + "customers.csv")
#pre_customers = pd.read_csv('Data/customers.csv')
print('-' * 80)
print('transactions')
pre_transactions = import_data(input_path_prefix + "transactions_train.csv")
#pre_transactions = pd.read_csv('Data/transactions_train.csv')

--------------------------------------------------------------------------------
articles
Memory usage of dataframe is 20.13 MB
Memory usage after optimization is: 7.25 MB
Decreased by 64.0%
--------------------------------------------------------------------------------
customers's memory usage will be better off without the memory optimization
--------------------------------------------------------------------------------
transactions
Memory usage of dataframe is 1212.63 MB
Memory usage after optimization is: 618.64 MB
Decreased by 49.0%


# Step 1.1 - Articles

# Step 1.2 - Customers

Customers data description:

* customer_id : A unique identifier of every customer
* FN : 1 or missed
* Active : 1 or missed
* club_member_status : Status in club
* fashion_news_frequency : How often H&M may send news to customer
* age : The current age
* postal_code : Postal code of customer

# Step 1.3 - Transactions

Transactions field description:

* t_dat : Purchase date
* day_of_week: Purchase day of week (Monday: 1, Tue: 2, …, Sun: 7) (This is a new field we added on top of the orignal dataset)
* customer_id : A unique identifier of every customer (in customers table)
* article_id : A unique identifier of every article (in articles table)
* price : Purchase price
* sales_channel_id : 1 or 2 (1 is store and 2 is online, ref: https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/305952#1684481) => It is transformed into sales_channel field with 1 is In-store and 2 is Online